In [ ]:
using Pkg
Pkg.activate("../tambo/")
using Tambo
using JLD2
using StatsBase
using Plots
using StaticArrays
using Glob 
using Measures
using Dierckx
using QuadGK
using CSV
using DataFrames

In [ ]:
TAMBO_PATH = "../Tambo"
# This should be the simulation parameters jld2 file

#path to sim file
SIMULATION_FILE = "/Users/pavelzhelnin/Documents/physics/TAMBO/resources/Larger_Valley/sim_files/larger_valley_00000_00023.jld2"

sim = jldopen(SIMULATION_FILE)
config = SimulationConfig(; geo_spline_path="../resources/tambo_spline.jld2", filter(x->x[1]!=:geo_spline_path, sim["config"])...)
geo = Tambo.Geometry(config)
injector = Tambo.Injector(config)
plane = Tambo.Plane(config.plane_orientation, config.tambo_coordinates,geo)
size = SVector{3}([2,2,0.03])*units.m
altmin = 2300units.m
altmax = 3700units.m
modules = Tambo.make_detector_array(
    5000units.m,
    150units.m,
    altmin,
    altmax,
    plane,
    geo,
    size,
)
nmodules = length(modules)

hard_SIMULATION_FILE = "/Users/pavelzhelnin/Documents/physics/TAMBO/resources/Larger_Valley/hard_spectrum/sim_files/larger_valley_00000_00023.jld2"
hard_sim = jldopen(hard_SIMULATION_FILE)
hard_config = SimulationConfig(; geo_spline_path="../resources/tambo_spline.jld2", filter(x->x[1]!=:geo_spline_path, hard_sim["config"])...)
hard_injector = Tambo.Injector(hard_config)

In [ ]:
#path to files 
file_names = ["retest_normal*.jld2","retest_small*jld2","retest_medium*jld2"]
num_files = 50
# for E^-2 spectrum
#simfiles = glob("larger*jld2","/Users/pavelzhelnin/Documents/physics/TAMBO/resources/Larger_Valley/sim_files")[1:num_files]
#files = glob(file_names[2],"/Users/pavelzhelnin/Documents/physics/TAMBO/resources/airshowers/big_showers/triggered_showers")[1:num_files]

# for E^-1 spectrum 
simfiles = glob("larger*jld2","/Users/pavelzhelnin/Documents/physics/TAMBO/resources/Larger_Valley/hard_spectrum/sim_files")[1:num_files]
files = glob(file_names[2],"/Users/pavelzhelnin/Documents/physics/TAMBO/resources/Larger_Valley/hard_spectrum/triggered_files")[1:num_files]

In [ ]:
function load_events(files)
    all_events = Vector{Tambo.InjectionEvent}()
    #det config has to be configured 
    for file in files
        triggered_events = load(file)["5000.0_150.0"]
        append!(all_events, triggered_events)
    end
    return all_events 
end 

In [ ]:
function combined_calc_triggered_event_rate(triggered_events; γ, ϕ, norm_e)
    norm = ϕ / units.GeV / units.cm^2 / units.second * (1 /(norm_e))^-γ
    pl = Tambo.PowerLaw(γ, 100units.GeV, 1e9units.GeV, norm)
    fluxes = pl.(triggered_events["initial_state"]["energy"])
    both_injectors = Vector{Tambo.Injector}([injector, hard_injector])
    both_injectors_xs = Vector{Tambo.CrossSection}([injector.xs, hard_injector.xs])
    wgts = [oneweight(triggered_event, both_injectors, both_injectors_xs) / (length(files)*config.n) for triggered_event in triggered_events]  # FIXME hack
    #wgts = oneweight.(triggered_events, Ref(injector), Ref(injector.xs)) ./ 20000
    global_fit_rates = fluxes .* wgts
    return global_fit_rates, wgts
end

In [ ]:
function spline_IC_bs()
    IC_names = ["IC_HESE_UP","IC_HESE_DOWN","IC_NU_MU_UP","IC_NU_MU_DOWN"]
    splines = [] 
    for name in IC_names 
        path = "/Users/pavelzhelnin/Downloads/$(name).csv"
        df = CSV.read(path,DataFrame)

        if name == "IC_HESE_DOWN"
            df = df[1:end-3,:]
        end 
        spl = Spline1D(log10.(df[:,1]),-log10.(df[:,2]))
        push!(splines,spl)
    end 
    return splines 
end 

In [ ]:
IC_splines = spline_IC_bs()

In [ ]:
function load_glashow() 
    Glashow_EA = [0.0, 0.0, 0.0, 0.0046816482044159495, 0.03713849798996347, 0.0739791408876635, 0.16322523978243142, 0.34027495622777654, 0.7334218313716241, 1.4473454167634539, 3.2348105831290495, 9.68887819318819, 37.94437758512714, 318.72949073974775, 5016.674414469627, 247.44682564367457, 97.07277582928188, 48.33559469984019, 26.2860132574856, 18.250634229930935, 18.512188665180652, 14.112834563170725, 8.066685692391541, 9.282281252955308, 1.4721553044072142, 6.056621969146591, 12.432071583190375, 0.02860029252080212, 2.198576437495593, 0.0, 3.4431868274079775, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    glashow_edges = 10 .^ (14:0.125:19)
    glashow_cents = (glashow_edges[1:end-1] .+ glashow_edges[2:end]) / 2
    Glashow_spl = Spline1D(log10.(glashow_cents[4:end-13]), log10.(Glashow_EA[4:end-13]))
    return Glashow_spl
end

In [ ]:
e1_files = glob(file_names[2],"/Users/pavelzhelnin/Documents/physics/TAMBO/resources/Larger_Valley/hard_spectrum/triggered_files")
e2_files = glob(file_names[2],"/Users/pavelzhelnin/Documents/physics/TAMBO/resources/airshowers/big_showers/triggered_showers")

e1_all_events = load_events(e1_files)
e2_all_events = load_events(e2_files)

e_all_events = append!(e1_all_events, e2_all_events)

e_triggered_events, e_wgts = combined_calc_triggered_event_rate(e_all_events; γ=2.52, ϕ=1.8e-18, norm_e=100units.TeV)

ten_year_rate = sum(e_triggered_events) * (5000/nmodules) * 10 * 10^7.5 * units.second 

display("New Valley Location: $(ten_year_rate)")

In [ ]:
pcolors = cgrad(:Purples,16,categorical=true,rev=true)
bcolors = cgrad(:Blues,16,categorical=true,rev=true)
pinkcolors = cgrad(:PiYG_9,16,categorical=true)[1:5]

In [ ]:
#edges = 10 .^ (14.5:1.0:18.5)
#edges = 10 .^ (14.9:0.2:17.1)
low_edges = 10 .^ (14.0:0.1:17.5)
high_edges = 10 .^ (15.0:0.1:18.5)
#cents = 10 .^ ((log10.(edges[1:end-1]) .+ log10.(edges[2:end])) / 2)
cents = 10 .^ (14.5:0.1:18)
norm_factors = Vector{Float64}([])
GW_norm_factors = Vector{Float64}([])
#for (idx,edge) in enumerate(edges[1:end-1])
diff_names = ["ICEHE","AUGER","ARIANNA","ARA","ANITAI_IV"]
energies = e_all_events["initial_state"]["energy"]
Glashow_spl = load_glashow()

#IC tau flux 
function IC_tau(x)
    if x < 3.16e14
        return 0 
    elseif x < 1e15
        return 65*(x/3.16e14)^0.72
    elseif x < 2e16
        return 150 * (x/1e15)^0.425
    else 
        return 0 
    end 
end


for (idx,edge) in enumerate(high_edges) 
    # println("edge: ", edge)
    #mask = edge .< energies .< edges[idx+1]
    mask = low_edges[idx] .< energies .< edge
    
    if length(energies[mask]) == 0
        println("no events in bin")
        continue
    end

    GR_norm = 1.0 * (1 /(cents[idx]))^-2
    GR_pl = Tambo.PowerLaw(-2, 100units.GeV, 1e9units.GeV, GR_norm)

    diff_rates = combined_calc_triggered_event_rate(e_all_events[mask]; γ=2, ϕ=1.0, norm_e=cents[idx])[1]
    GR_rates = quadgk(x-> 10 .^Glashow_spl(log10(x)) * 1.0 * (x /(cents[idx]))^-2 * 1e4 * 1e-9, low_edges[idx],edge)[1] 
    IC_tau_rates = quadgk(x-> IC_tau(x) * 1.0 * (x /(cents[idx]))^-2 * 1e4 * 1e-9, low_edges[idx],edge)[1]
    #rw_diff_rates, _ = quadgk(e-> diff_integrand(e, cents[idx]/1e9, 2), low_edges[idx]/1e9, edge/1e9)
    
    #push!(rw_norm_factors, 2.44 / (rw_diff_rates* 10 * 10^7.5) ) 
    #FC 90% sensitivity = 2.44 events 
    #norm_factor = 2.44 /  sum(diff_rates * (5000/nmodules) * 10 * 10^7.5 * units.second)
    norm_factor = 2.44 / (IC_tau_rates * 10 * 10^7.5)
    GW_norm_factor = 2.44 / (((sum(diff_rates) * units.second * (5000/nmodules)) + (1/2 * GR_rates)) * 10 * 10^7.5)

    push!(norm_factors,norm_factor)
    push!(GW_norm_factors,GW_norm_factor)
end
#display(rw_norm_factors)
GW_diff_sens = GW_norm_factors
diff_sens = norm_factors 

#display(rw_diff_sens)
plt = plot(
    xscale=:log,
    yscale=:log,
    xlabel="Neutrino energy [GeV]",
    xminorticks=true,
    yminorticks=true,
    ylabel="\$ \\textnormal{\\textit{E}^{2}\\Phi_{\\nu + \\bar{\\nu}} [GeV cm^{-2} s^{-1} sr^{-1}]}\$",
    xlimits=(1e5, 1e9),
    title="",
    ylimits=(1e-10,1e-5),
    legend=:topleft,
    framestyle=:box,
    grid=false,
    tickfontsize=12,
    #palette= cgrad(:BuPu_9,16,categorical=true,rev=true)[1:2:9],
    # left_margin=5mm,
    # right_margin=5mm,
    bottom_margin=2mm,
    fontfamily="Computer Modern",
    foreground_color_legend = nothing,
    size=(600, 400)   
)

# plot!(
#     plt,
#     #edges / units.GeV,
#     cents / units.GeV,
#     #log.(10, (cents / units.GeV).^ 2 .* diff_sens),
#     (cents / units.GeV).^ 2 .* diff_sens,
#     # hist_three.weights .*norm_factors,
#     #linetype=:stepbins,
#     color=pcolors[3],
#     linestyle=:dash,
#     linetype=:line,
#     label="TAMBO",
#     lw=3*1.25
#     )

plot!(
    plt,
    #edges / units.GeV,
    cents / units.GeV,
    #log.(10, (cents / units.GeV).^ 2 .* diff_sens),
    (cents / units.GeV).^ 2 .* GW_diff_sens,
    # hist_three.weights .*norm_factors,
    #linetype=:stepbins,
    #color=pcolors[7],
    color=pcolors[3],
    linestyle=:dash,
    linetype=:line,
    label="TAMBO",
    lw=3*1.25
    )

# plot!(
#     plt,
#     #edges / units.GeV,
#     cents / units.GeV,
#     #log.(10, (cents / units.GeV).^ 2 .* diff_sens),
#     (cents / units.GeV).^ 2 .* GW_diff_sens .* (5000/100),
#     # hist_three.weights .*norm_factors,
#     #linetype=:stepbins,
#     #color=pcolors[7],
#     color=pcolors[3],
#     linestyle=:dot,
#     linetype=:line,
#     label="TAMBITO",
#     lw=3*1.25
#     )

# plot!(
#     plt,
#     #edges / units.GeV,
#     cents / units.GeV,
#     #log.(10, (cents / units.GeV).^ 2 .* diff_sens),
#     (cents / units.GeV).^ 2 .* GW_diff_sens .* (5000/20000),
#     # hist_three.weights .*norm_factors,
#     #linetype=:stepbins,
#     #color=pcolors[7],
#     color=pcolors[3],
#     linestyle=:dashdot,
#     linetype=:line,
#     label="TAMBISSIMO",
#     lw=3*1.25
#     )
IC_min = 2.5e13
IC_max = 5e15 
IC_cents = 10 .^ (log10(IC_min):0.1:log10(IC_max))
plot!(
    plt, 
    IC_cents / units.GeV,
    10 .^-IC_splines[2].(log10.(IC_cents));
    fillrange = 10 .^-IC_splines[1].(log10.(IC_cents)),
    fillalpha=0.35,
    c=bcolors[8],
    label="",
    lw=1
)

plot!(
    plt, 
    IC_cents / units.GeV,
    10 .^-IC_splines[1].(log10.(IC_cents)),
    c=bcolors[8],
    label="",
    lw=1
)

IC_min = 2.5e13
IC_max = 2.75e15 
IC_cents = 10 .^ (log10(IC_min):0.1:log10(IC_max))
plot!(
    plt, 
    IC_cents / units.GeV,
    10 .^-IC_splines[4].(log10.(IC_cents));
    fillrange = 10 .^-IC_splines[3].(log10.(IC_cents)),
    fillalpha=0.35,
    c=bcolors[4],
    label="",
    lw=1
)

plot!(
    plt, 
    IC_cents / units.GeV,
    10 .^-IC_splines[3].(log10.(IC_cents)),
    c=bcolors[4],
    label="",
    lw=1
)

for (idx,name) in enumerate(diff_names)
    path = "/Users/pavelzhelnin/Downloads/$(name).csv"
    df = CSV.read(path,DataFrame)
    if name == "ANITAI_IV"
        name = "ANITA I - IV"
        continue 
    end 

    color = pinkcolors[idx+1]
    
    if name == "ICEHE"
        name = "IC EHE"
        color = bcolors[7]
    end

    if name == "ARA"
        color = pinkcolors[idx+1]
    end

    # if name == "AUGER"
    #     color = cgrad(:BuPu_9,9,categorical=true,rev=true)[8]
    #     plot!(plt, df[:,1][1:2:end]./1e9, df[:,2][1:2:end], color=color,label=name, lw=3)
    #     continue 
    # end 

    plot!(plt, df[:,1][1:2:end]./1e9, df[:,2][1:2:end], color =color, label=name, lw=3)
end

# plot!(
#         plt,
#         #edges / units.GeV,
#         cents / units.GeV,
#         #log.(10, (cents / units.GeV).^ 2 .* diff_sens),
#         (cents / units.GeV).^ 2 .* rw_diff_sens,
#         # hist_three.weights .*norm_factors,
#         #linetype=:stepbins,
#         linetype=:line,
#         label="redid SnowMass paper in Julia today",
#         lw=3
#         )

# csv_energies = [1.00E+06,1.58E+06,2.51E+06,3.98E+06,6.31E+06,1.00E+07,1.58E+07,2.51E+07,3.98E+07,6.31E+07,1.00E+08]
# csv_diff_sens = [2.57E-09,1.90E-09,1.47E-09,1.18E-09,9.96E-10,8.81E-10,8.10E-10,7.74E-10,7.73E-10,8.32E-10,9.58E-10]

# plot!(
#     plt,
#     csv_energies,
#     csv_diff_sens,
#     linetype=:line,
#     label="0.1 \$ \\times \$ SnowMass paper",
#     lw=3
#     )


# plot!(
#     plt,
#     cents/units.GeV,
#     (cents/units.GeV).^2 .* diff_sens,
#     color=:green,
#     linetype=:line,
#     label="IC \$ \\nu_{\\tau} \$",
#     lw=3*1.25,)

#copy_ticks(current(),([0,π/2,π,3π/2,2π],["0","π/2","π","3π/2","2π"]))
# energy_ticks = [5e5,5e6,5e7,5e8]
# CoM_ticks = ["10^{$i}" for i in round.(log10.(sqrt.(2*energy_ticks)),digits=3)]
# copy_ticks(current(),(energy_ticks,CoM_ticks))
display(plt)
savefig(plt,"new_TAMBO_diff_sens.svg")

In [ ]:
function twiny(sp::Plots.Subplot)
    sp[:top_margin] = max(sp[:top_margin], 30Plots.px)
    plot!(sp.plt, inset = (sp[:subplot_index], bbox(0,0,1,1)))
    twinsp = sp.plt.subplots[end]
    twinsp[:xaxis][:mirror] = true
    twinsp[:background_color_inside] = RGBA{Float64}(0,0,0,0)
    Plots.link_axes!(sp[:yaxis], twinsp[:yaxis])
    twinsp
end
twiny(plt::Plots.Plot = current()) = twiny(plt[1])

function copy_ticks(sp::Plots.Subplot,xticks)
    ptx = twinx(sp)
    plot!(ptx,xlims=xlims(plt),ylims=ylims(plt),
    #xticks=xticks,
    xlabel="",
    yformatter=_->"",
    xformatter=_->"",
    #xminorticks=true,
    yminorticks=true,
    yscale=:log10,
    xscale=:log10,
    grid=false)
    pty = twiny(sp)
    plot!(pty,xlims=xlims(plt),ylims=ylims(plt),label="",
    yformatter=_->"",
    #xminorticks=true,
    xscale=:log10, 
    tickfontsize=12,
    xticks=xticks,
    fontfamily="Computer Modern",
    xlabel="\$ \\sqrt{s} \\ \\textnormal{[GeV]} \$",
    grid=false)
end

copy_ticks(plt::Plots.Plot, xticks) = copy_ticks(plt[1],xticks)
plt = plot(sin, xlims=(0+0.1,2π),label="",grid=false,xscale=:log10)
copy_ticks(current(),([1,10],[1,10]))
display(plt)

In [ ]:
function twiny(sp::Plots.Subplot)
    sp[:top_margin] = max(sp[:top_margin], 30Plots.px)
    plot!(sp.plt, inset = (sp[:subplot_index], bbox(0,0,1,1)))
    twinsp = sp.plt.subplots[end]
    twinsp[:xaxis][:mirror] = true
    twinsp[:background_color_inside] = RGBA{Float64}(0,0,0,0)
    Plots.link_axes!(sp[:yaxis], twinsp[:yaxis])
    twinsp
end
twiny(plt::Plots.Plot = current()) = twiny(plt[1])

function copy_ticks(sp::Plots.Subplot,xticks)
    ptx = twinx(sp)
    plot!(ptx,xlims=xlims(plt),ylims=ylims(plt),yformatter=_->"",xformatter=_->"", grid=false)
    pty = twiny(sp)
    plot!(pty,xlims=xlims(plt),ylims=ylims(plt),yformatter=_->"",xticks=xticks, grid=false)
end
copy_ticks(plt::Plots.Plot,xticks) = copy_ticks(plt[1],xticks)

plt = plot(sin, xlims=(0,2π))
copy_ticks(current(),([0,π/2,π,3π/2,2π],["0","π/2","π","3π/2","2π"]))
display(plt)

In [ ]:
#edges = 10 .^ (14.5:1.0:18.5)
#edges = 10 .^ (14.9:0.2:17.1)
low_edges = 10 .^ (14.0:0.1:17.5)
high_edges = 10 .^ (15.0:0.1:18.5)
#cents = 10 .^ ((log10.(edges[1:end-1]) .+ log10.(edges[2:end])) / 2)
cents = 10 .^ (14.5:0.1:18)
norm_factors = Vector{Float64}([])
GW_norm_factors = Vector{Float64}([])
#for (idx,edge) in enumerate(edges[1:end-1])
diff_names = ["ICEHE","AUGER","ARA","ARIANNA","ANITAI_IV"]
energies = e_all_events["initial_state"]["energy"]
Glashow_spl = load_glashow()
bg_rates =[]
for (idx,edge) in enumerate(high_edges) 
    # println("edge: ", edge)
    #mask = edge .< energies .< edges[idx+1]
    mask = low_edges[idx] .< energies .< edge
    
    if length(energies[mask]) == 0
        println("no events in bin")
        continue
    end

    GR_norm = 1.0 * (1 /(cents[idx]))^-2
    GR_pl = Tambo.PowerLaw(-2, 100units.GeV, 1e9units.GeV, GR_norm)

    #diff_rates = combined_calc_triggered_event_rate(e_all_events[mask]; γ=2.52, ϕ=1.8e-18, norm_e=cents[idx])[1]
    diff_rates = combined_calc_triggered_event_rate(e_all_events[mask]; γ=2.52, ϕ=1.8e-18, norm_e=100units.TeV)[1]
    #GR_rates = quadgk(x-> 10 .^Glashow_spl(log10(x)) * 1.0 * (x /(cents[idx]))^-2. * 1e4 * 1e-9, low_edges[idx],edge)[1] 
    GR_rates = quadgk(x-> 10 .^Glashow_spl(log10(x)) * (1.8e-18) * (x /(100units.TeV))^-2.52 * 1e4 * 1e-9, low_edges[idx], edge)[1] 
    #display(GR_rates)
    #rw_diff_rates, _ = quadgk(e-> diff_integrand(e, cents[idx]/1e9, 2), low_edges[idx]/1e9, edge/1e9)
    
    #push!(rw_norm_factors, 2.44 / (rw_diff_rates* 10 * 10^7.5) ) 
    #FC 90% sensitivity = 2.44 events 
    #norm_factor = 2.44 /  sum(diff_rates * (5000/nmodules) * 10 * 10^7.5 * units.second)
    #GW_norm_factor = 2.44 / (((sum(diff_rates) * units.second * (5000/nmodules)) + (1/2 * GR_rates)) * 10 * 10^7.5)
    
    #converting from steradians to square degrees, integrating from 1 deg^2 patch of sky 
    bg_rate = ((sum(diff_rates) * units.second * (5000/nmodules)) + (1/2 * GR_rates)) * 10 * 10^7.5 * (units.degree).^2

    push!(bg_rates,bg_rate)
    #what is the 90% FC upper limit on true signal given we observed 1 event 
    n0s = zeros(1)
    #upper_limit = lowest_n_splines[n0+1](bg_rate) 
    lower_limit = zeros(1)
    # for n0 in 0:1:6
    #     lower_limit[1]= highest_n_splines[n0+1](bg_rate)
    #     if lower_limit[1] > 0.001
    #         #display(n0)
    #         n0s[1] = n0 
    #         break
    #     end 
    # end 

    lower_limit = zeros(1)
    μs = 0.

    average = 0.
    for n0 in 0:1:9
        lower_limit[1]= highest_n_splines[n0+1](bg_rate)
        upper_limit = lowest_n_splines[n0+1](bg_rate)
        x = Poisson(bg_rate)
        μ = pdf(x,n0)
        μs += μ
        average += μ * upper_limit
    end 
    lower_limit = average/μs
    #lower_limit = lower_limit[1]
    diff_rates = combined_calc_triggered_event_rate(e_all_events[mask]; γ=2, ϕ=1., norm_e=cents[idx])[1]
    GR_rates = quadgk(x-> 10 .^Glashow_spl(log10(x)) * 1.0 * (x /(cents[idx]))^-2. * 1e4 * 1e-9, low_edges[idx],edge)[1] 
    #lower_limit = 2.44 
    lower_limit = 1.
    GW_norm_factor = lower_limit / (((sum(diff_rates) * units.second * (5000/nmodules)) + (1/2 * GR_rates)) * 10 * 10^7.5)

    #push!(norm_factors,norm_factor)
    push!(GW_norm_factors,GW_norm_factor)
end

plt = plot(
    xscale=:log,
    yscale=:log,
    xlabel="Neutrino energy [TeV]",
    xminorticks=true,
    yminorticks=true,
    ylabel="\$E^{2}\\Phi_{\\nu + \\bar{\\nu}}\$  \$ \\textnormal{[TeV cm^{-2} s^{-1}]}\$",
    xlimits=(1e2, 1e7),
    title="TAMBO single event no background differential sensitivity",
    ylimits=(1e-12,1e-8),
    legend=:topleft,
    framestyle=:box,
    tickfontsize=12,
    #palette= cgrad(:BuPu_9,16,categorical=true,rev=true)[1:2:9],
    # left_margin=5mm,
    right_margin=5mm,
    top_margin=5mm,
    # bottom_margin=5mm,
    fontfamily="Computer Modern",
    foreground_color_legend = nothing,
    size=(600, 400)   
)

plot!(
    plt,
    #edges / units.GeV,
    cents / units.TeV,
    #log.(10, (cents / units.GeV).^ 2 .* diff_sens),
    (cents / units.TeV).^ 2 .* GW_norm_factors * 10^3 .* (100*units.degree)^2,
    #(cents / units.GeV).^ 2 .* GW_norm_factors * 10^-3 .* 4pi,
    # hist_three.weights .*norm_factors,
    #linetype=:stepbins,
    color=pcolors[7],
    linestyle=:dash,
    linetype=:line,
    label="TAMBO",
    lw=3*1.25
    )
#savefig(plt,"TAMBO_single_event_diff_sens.pdf")
display(plt)


In [ ]:
(cents / units.TeV).^ 2 .* GW_norm_factors * 10^3 .* (100*units.degree)^2

In [ ]:
y = 2.29
test_y = Poisson(sum(bg_rates)+y)
cumsum(pdf(test_y,0:1:10))

In [ ]:
sum(bg_rates)+y

In [ ]:
units.degree^-2

In [ ]:
test_x = Poisson(2.44)

In [ ]:
cumsum(pdf(test_x,0:1:2))

In [ ]:
iszero(highest_n_splines[2](2.5))

In [ ]:
colors = cgrad(:BuPu_9,16,categorical=true,rev=true)

In [ ]:
pcolors = cgrad(:Purples,16,categorical=true,rev=true)

In [ ]:
bcolors = cgrad(:Blues,16,categorical=true,rev=true)

In [ ]:
pinkcolors = cgrad(:PiYG_9,16,categorical=true)[1:5]

In [ ]:
using Distributions
function poisson_prob(s,b,n)
    return (s+b)^n * exp(-(s+b)) / factorial(big(n))
end
function R(s,b,n)
    μ = max(0,n-b)
    return poisson_prob(s,b,n) / poisson_prob(μ,b,n)
end

In [ ]:
b = 3.
ss = 0:0.01:15
ns = 25:-1:0

n_UL_lows = [] 
nss = []

for s in ss
    probs = poisson_prob.(s,b,ns)
    prob_mask = cumsum(probs) .<= 0.90 
    prob_mask[findall(x -> x == 0, prob_mask)[1]] = true
    push!(nss,ns[prob_mask])
    push!(n_UL_lows,minimum(ns[prob_mask]))
end 

In [ ]:
maximum(GW_norm_factors)

In [ ]:
function make_FC_spline(signal_scan,background_scan,n,n0,α)
    #n0 returns the number of observed signal you're concerned about 
    lowest_n = zeros(Float64,length(background_scan))
    highest_n = zeros(Float64,length(background_scan))
    for (i,b) in enumerate(background_scan)
        n_lows = zeros(Int64,length(signal_scan))
        n_highs = zeros(Int64,length(signal_scan))
        for (j,s) in enumerate(signal_scan)
            Rs = R.(s,b,n)
            poisson_pdf = Poisson(s+b)
            probs = pdf.(poisson_pdf,n) 
            mask = sortperm(Rs,rev=true)
            prob_mask = cumsum(probs[mask]) .<= α
            try 
                prob_mask[findall(x -> x == 0, prob_mask)[1]] = true
            catch 
                println(cumsum(probs[mask]))
                println("prob mask issue")
                println(prob_mask)
                return prob_mask
            end 
            n_lows[j] = minimum(n[mask][prob_mask])
            n_highs[j] = maximum(n[mask][prob_mask])
        end
        try
            lowest_n[i] = signal_scan[findlast(x->x==n0,n_lows)]
        catch 
            println("lowest_n issue")
            println("n0: $n0")
            println("b: $b")
            println(n_lows)
            println(signal_scan)
            return 0
        end 
        try 
            highest_n[i] = signal_scan[findfirst(x->x==n0,n_highs)]
        catch
            highest_n[i] = 0. 
        end 
    end 
    lowest_n, highest_n = FC_correction(lowest_n,highest_n)
    return Spline1D(background_scan,lowest_n), Spline1D(background_scan,highest_n)
end 

In [ ]:
function FC_correction(lowest_n,highest_n)
    all_fixed = false

    while !all_fixed
        all_fixed = true
        for j in 2:length(lowest_n)  # Julia arrays are 1-based, so we start from 2
            if lowest_n[j] > lowest_n[j - 1]
                lowest_n[j - 1] = lowest_n[j]
                all_fixed = false
            end
        end

        for j in 2:length(highest_n)  # Julia arrays are 1-based, so we start from 2
            if highest_n[j] > highest_n[j - 1]
                highest_n[j - 1] = highest_n[j]
                all_fixed = false
            end
        end
    end

    return lowest_n, highest_n
end

In [ ]:
!iszero(highest_n_splines[1](4))

In [ ]:
ss = 0:0.005:16
ns = 0:1:25
bs = 0:0.005:3
α = 0.9
lowest_n_splines = [] 
highest_n_splines = []
for n0 in 0:1:9
    push!(lowest_n_splines,make_FC_spline(ss,bs,ns,n0,α)[1])
    push!(highest_n_splines,make_FC_spline(ss,bs,ns,n0,α)[2])
end 
save("FC_splines.jld2","lowest_n_splines",lowest_n_splines,"highest_n_splines",highest_n_splines)

In [ ]:
using JLD2

In [ ]:
save("FC_splines.jld2","lowest_n_splines",lowest_n_splines,"highest_n_splines",highest_n_splines)

In [ ]:
splines = load("FC_splines.jld2")

In [ ]:
splines["lowest_n_splines"][1](1.5)

In [ ]:
ghd

In [ ]:
ghd = Poisson(16+3)
cumsum(pdf.(ghd,0:1:24))

In [ ]:
display(lowest_n_splines)

In [ ]:
ss = 0:0.005:20
ns = 0:1:15
bs = 0:0.005:5
α = 0.9
lowest_n_splines = [] 
highest_n_splines = []
for n0 in 0:1:6
    push!(lowest_n_splines,make_FC_spline(ss,bs,ns,n0,α)[1])
    push!(highest_n_splines,make_FC_spline(ss,bs,ns,n0,α)[2])
end 

In [ ]:
for n in 5:1:5
    println(n)
end

In [ ]:
#b = 4.0
ss = 0:0.005:8
ns = 0:1:20
lowest_n = [] 
highest_n = []
for b in 0:0.1:5
#for b in GW_norm_factors
    n_lows = [] 
    n_highs = []
    nss = []
    for s in ss
        Rs = R.(s,b,ns)
        probs = poisson_prob.(s,b,ns)
        mask = sortperm(Rs,rev=true)
        prob_mask = cumsum(probs[mask]) .<= 0.90 
        prob_mask[findall(x -> x == 0, prob_mask)[1]] = true

        push!(nss,ns[mask][prob_mask])
        push!(n_lows,minimum(ns[mask][prob_mask]))
        push!(n_highs,maximum(ns[mask][prob_mask]))
    end 
    try 
        push!(lowest_n, ss[findlast(x->x==4,n_lows)])
    catch 
        display(n_lows)
        display(b)
        break
    end
    try 
        push!(highest_n, ss[findfirst(x->x==4,n_highs)])
    catch 
        # display(n_highs)
        # display(b)
        # break
        push!(highest_n, 0)
    end
end 

In [ ]:
bs = 1:0.1:5

In [ ]:
all_fixed = false

while !all_fixed
    all_fixed = true
    for j in 2:length(highest_n)  # Julia arrays are 1-based, so we start from 2
        if highest_n[j] > highest_n[j - 1]
            highest_n[j - 1] = highest_n[j]
            all_fixed = false
        end
    end
end

In [ ]:
n_lows = [] 
n_highs = []
nss = []
b = 4.0
ss = 0.:0.005:15.
#ss = [0.745, 0.750, 0.755]
#ss = [0.74, 0.75, 0.76, 1.03, 1.04, 1.05, 1.06, 1.07]
ss = [0.84,0.85,0.86,1.05,1.01,1.015]
ns = 0:1:20
for s in ss
    Rs = R.(s,b,ns)
    probs = poisson_prob.(s,b,ns)
    mask = sortperm(Rs,rev=true)
    prob_mask = cumsum(probs[mask]) .<= 0.93
    try 
        prob_mask[findall(x -> x == 0, prob_mask)[1]] = true
    catch 
        continue
    end

    push!(nss,ns[mask][prob_mask])
    push!(n_lows,minimum(ns[mask][prob_mask]))
    push!(n_highs,maximum(ns[mask][prob_mask]))
end 
nss

In [ ]:
ss[findlast(x->x==0,n_lows)]

In [ ]:
cumsum(poisson_prob.(ss[101],b,nss[101]))

In [ ]:
ss[findlast(x->x==0,n_lows)]

In [ ]:
p = Poisson(b)

nn = [] 
for n in rand(p,1000)
    if n == 0 
        push!(nn,0.88)
    else 
        push!(nn,ss[findlast(x->x==n,n_lows)])
    end 
end

In [ ]:
sum(nn)/1000

In [ ]:
plot(n_lows,ss,label="",xlabel="n",xticks=0:1:15,ylabel="s",yticks=0:1:15,xlimits=(0,15),aspect_ratio=:equal)
plot!(n_highs,ss,label="")
plot!(n_UL_lows,ss,label="")
hline!([1],label="")

In [ ]:
sum(poisson_prob.(0,3,0:1:5) .* [1.08,1.88,3.04,4.42,5.6,6.99]) / sum(poisson_prob.(0,3,0:1:5))

In [ ]:
signals = [1.61,3.36,4.91,6.42,7.6,8.99,10.47,11.53]

In [ ]:
bg_rate = 1.

average = 0.
μs = 0.
for i in 0:1:7
    #ul_signal = lowest_n_splines[i](bg_rate)
    ul_signal = signals[i+1]
    μ = poisson_prob(0,bg_rate,i) 
    μs += μ
    average += μ * ul_signal
end 
display(average/μs)

In [ ]:
lower_limit = zeros(1)
bg_rate = 1.0 
μs = 0.

average = 0.
for n0 in 0:1:6
    lower_limit[1]= highest_n_splines[n0+1](bg_rate)
    # if lower_limit[1] > 0.001
    #     break
    # end 
    display(lower_limit)
    upper_limit = lowest_n_splines[n0+1](bg_rate)
    x = Poisson(bg_rate)
    μ = pdf(x,n0)
    μs += μ
    average += μ * upper_limit
end 
display(average/μs)